In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
!pip install efficientnet_pytorch torchtoolbox

import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
import torchtoolbox.transform as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, GroupKFold
import pandas as pd
import numpy as np
import gc
import os
import cv2
import time
import datetime
import warnings
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
%matplotlib inline

In [ ]:
class AdvancedHairAugmentation:
    """
    Impose an image of a hair to the target image

    Args:
        hairs (int): maximum number of hairs to impose
        hairs_folder (str): path to the folder with hairs images
    """

    def __init__(self, hairs: int = 5, hairs_folder: str = ""):
        self.hairs = hairs
        self.hairs_folder = hairs_folder

    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to draw hairs on.

        Returns:
            PIL Image: Image with drawn hairs.
        """
        n_hairs = random.randint(0, self.hairs)
        
        if not n_hairs:
            return img
        
        height, width, _ = img.shape  # target image width and height
        hair_images = [im for im in os.listdir(self.hairs_folder) if 'png' in im]
        
        for _ in range(n_hairs):
            hair = cv2.imread(os.path.join(self.hairs_folder, random.choice(hair_images)))
            hair = cv2.flip(hair, random.choice([-1, 0, 1]))
            hair = cv2.rotate(hair, random.choice([0, 1, 2]))

            h_height, h_width, _ = hair.shape  # hair image width and height
            roi_ho = random.randint(0, img.shape[0] - hair.shape[0])
            roi_wo = random.randint(0, img.shape[1] - hair.shape[1])
            roi = img[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width]

            # Creating a mask and inverse mask
            img2gray = cv2.cvtColor(hair, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)

            # Now black-out the area of hair in ROI
            img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

            # Take only region of hair from hair image.
            hair_fg = cv2.bitwise_and(hair, hair, mask=mask)

            # Put hair in ROI and modify the target image
            dst = cv2.add(img_bg, hair_fg)

            img[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width] = dst
                
        return img

    def __repr__(self):
        return f'{self.__class__.__name__}(hairs={self.hairs}, hairs_folder="{self.hairs_folder}")'

In [ ]:
class MultimodalDataset(Dataset):
    """
    Custom dataset definition
    """
    def __init__(self, csv_path, img_path, transform=None):
        """
        """
        self.df = pd.read_csv(csv_path)
        self.img_path = img_path
        self.transform = transform
        
            
    def __getitem__(self, index):
        """
        """
        img_name = self.df.iloc[index]["image_name"] 
        img_path = os.path.join(self.img_path, img_name)
        image = Image.open(img_path)
        if self.df.iloc[index]["augmented"]==1{
            image = AdvancedHairAugmentation()(image)
        }
        image = image.convert("RGB")
        image = np.asarray(image)
        if self.transform is not None:
            image = self.transform(image)
            
        dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor # ???
        features = np.fromstring(self.df.iloc[index]["features"][1:-1], sep=",") #turns features into an array
        features = torch.from_numpy(features.astype("float")) #turns the features array into a vector
        #label = int(self.df.iloc[index]['label'])
        labels = torch.tensor(list(self.df.iloc[index]["target"]), dtype = torch.float64)
        #print("Label type: ", type(label))
        #label = np.int_(label) #???
        #print("label type post casting: ", type(label))
        return image, features, labels
        
    def __len__(self):
        return len(self.df)

In [ ]:
def get_dataloaders(input_size, batch_size, augment=False, shuffle = True):
    # How to transform the image when you are loading them.
    # you'll likely want to mess with the transforms on the training set.
    
    # For now, we resize/crop the image to the correct input size for our network,
    # then convert it to a [C,H,W] tensor, then normalize it to values with a given mean/stdev. These normalization constants
    # are derived from aggregating lots of data and happen to produce better results.
    data_transforms = {
        'train': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            #Add extra transformations for data augmentation
            AdvancedHairAugmentation(hairs_folder="/kaggle/input/melanoma-hairs/")
            transforms.RandomApply([
                transforms.RandomChoice([
                    transforms.RandomAffine(degrees=20),
                    transforms.RandomAffine(degrees=0,scale=(0.1, 0.15)),
                    transforms.RandomAffine(degrees=0,translate=(0.2,0.2)),
                    #transforms.RandomAffine(degrees=0,shear=0.15),
                    transforms.RandomHorizontalFlip(p=1.0)
                ] if augment else [transforms.RandomAffine(degrees=0)])#else do nothing
            ], p=0.5),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ]),
        #'val': transforms.Compose([
            #transforms.ToPILImage(),
            #transforms.Resize(input_size),
            #transforms.CenterCrop(input_size),
            #transforms.ToTensor(),
            #transforms.Normalize([0.5], [0.225])
        #]),
        'test': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ])
    }
    # Create training and validation datasets
    data_subsets = {x: MultimodalDataset(csv_path="../input/melanoma/features_"+x+".csv", 
                                         img_path="../input/siim-isic-melanoma-classification/jpeg/"+x, 
                                         transform=data_transforms[x]) for x in data_transforms.keys()}
    print(data_transforms[0])
    # Create training and validation dataloaders
    # Never shuffle the test set
    dataloaders_dict = {x: DataLoader(data_subsets[x], batch_size=batch_size, shuffle=False if x != 'train' else shuffle, num_workers=4) for x in data_transforms.keys()}
    return dataloaders_dict

In [ ]:
shuffle_datasets = True
dataloaders = get_dataloaders(224, 64, shuffle_datasets)
dataloaders